# **5 Day Weather Forecast**

## 1. Setup

In [1]:
# 1. Setup & Imports
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import re
import os

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import shap
from statsmodels.graphics.tsaplots import plot_acf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    mean_absolute_percentage_error,
    make_scorer,
    mean_squared_error,
    root_mean_squared_error,
    mean_absolute_error,
    r2_score
)
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.feature_selection import mutual_info_regression

from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from sklearn.ensemble import StackingRegressor


In [2]:
def rmse(y, yhat):
    return np.sqrt(mean_squared_error(y, yhat))

## 2. Load local weather data

In [3]:
df = pd.read_excel("HCMWeatherDaily.xlsx")
df.head(5)


,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"10.82, 106.67",2015-01-01,31.0,23.0,26.6,31.3,23.0,26.9,17.9,60.4,...,19.9,8,NaN,2015-01-01T06:11:34,2015-01-01T17:41:42,0.36,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"48894099999,48900099999,VVTS"
1,"10.82, 106.67",2015-01-02,30.0,20.0,25.0,30.3,20.0,25.1,15.7,56.7,...,16.1,7,NaN,2015-01-02T06:11:58,2015-01-02T17:42:15,0.39,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"48894099999,48900099999,VVTS"
2,"10.82, 106.67",2015-01-03,32.0,23.0,26.7,33.4,23.0,27.4,18.9,63.3,...,16.0,8,NaN,2015-01-03T06:12:21,2015-01-03T17:42:47,0.43,"Rain, Partially cloudy",Partly cloudy throughout the day with rain in ...,rain,"48894099999,48900099999,VVTS"
3,"10.82, 106.67",2015-01-04,32.0,24.0,27.1,34.8,24.0,28.3,21.2,71.3,...,10.9,4,NaN,2015-01-04T06:12:44,2015-01-04T17:43:20,0.46,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"48894099999,48900099999,VVTS"
4,"10.82, 106.67",2015-01-05,30.9,25.0,26.7,33.6,25.0,27.8,22.0,76.3,...,14.1,8,NaN,2015-01-05T06:13:05,2015-01-05T17:43:53,0.50,"Rain, Partially cloudy",Partly cloudy throughout the day with morning ...,rain,"48894099999,48900099999,VVTS"


## 3. Basic cleaning and encoding

In [4]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['sunrise'] = pd.to_datetime(df['sunrise'])
df['sunset'] = pd.to_datetime(df['sunset'])

df.drop(columns = ['name','snow','snowdepth','conditions','description','icon','stations','severerisk'], inplace = True)

In [5]:
# --- One-hot encode preciptype to has_rain (rain=1, NaN/others=0)
df['has_rain'] = (
    df['preciptype']
    .fillna('')          
    .str.lower()         
    .str.contains('rain')
    .astype(int)        
)

# Optional: drop the original column
df.drop(columns=['preciptype'], inplace=True)

In [6]:
# Precipitation (in → mm)
df['precip'] = df['precip'] * 25.4

# Percentage-based to fraction
pct_cols = ['humidity','cloudcover','precipprob','precipcover']
df[pct_cols] = df[pct_cols] / 100.0

# Wind (mph → m/s)
wind_cols = ['windspeed','windgust']
df[wind_cols] = df[wind_cols] / 2.237

# Visibility (mi → km)
df['visibility'] = df['visibility'] * 1.609

# Convert MJ/m²/day → W/m²-equivalent
df['solarenergy_wm2eq'] = df['solarenergy'] * 11.6

In [7]:
df['day_length'] = (
    (df['sunset'] - df['sunrise'])
    .dt.total_seconds()
    .div(3600)
    .clip(lower=0, upper=24)
)
df.drop(columns = ['sunrise','sunset'], inplace = True)

## 4. Physics‑based feature engineering

In [8]:
# --- PHYSICS-BASED FEATURES (NO TEMP VARIABLES) ------------------------

# 1. Radiation efficiency — solar input vs cloud attenuation
df['radiation_efficiency'] = (
    (df['solarradiation'] + 0.1 * df['solarenergy'])
    / (df['cloudcover'].clip(lower=1) + 1e-3)
)
# assuming solarenergy is MJ/m2/day
df['rad_per_hour'] = df['solarenergy'] / (df['day_length'] + 1e-3)


# 2. Moisture ratio — dew point relative to humidity
df['dew_humidity_ratio'] = df['dew'] / df['humidity'].replace(0, np.nan)
df['dew_humidity_ratio'] = df['dew_humidity_ratio'].fillna(0)

# 3. Precipitation intensity — how concentrated rainfall is
df['precip_intensity'] = (
    df['precip'] / (df['precipcover'].replace(0, np.nan) + 1e-3)
).clip(upper=100)  # avoid division blow-ups

# 4. Wind components (convert polar → Cartesian)
df['wind_u'] = df['windspeed'] * np.cos(np.deg2rad(df['winddir']))
df['wind_v'] = df['windspeed'] * np.sin(np.deg2rad(df['winddir']))

# 5. Mean-centering to remove directional bias (optional for SHAP clarity)
df['wind_u'] -= df['wind_u'].mean()
df['wind_v'] -= df['wind_v'].mean()

# 6. Storminess index — dynamic proxy for convective activity
df['storminess'] = (df['windspeed'] ** 2) * (df['precipprob'] / 100.0)

# 7. Seasonal cycle — encode year-round periodicity
df['dayofyear'] = df['datetime'].dt.dayofyear
df['doy_sin'] = np.sin(2 * np.pi * df['dayofyear'] / 365.25)
df['doy_cos'] = np.cos(2 * np.pi * df['dayofyear'] / 365.25)

## 5. Features engineering

In [9]:
windows = [3, 7, 14, 28, 56]
df = df.sort_values('datetime').reset_index(drop=True)

# ---------------------------------------------------------------
# 1️⃣  ROLLING /  LAGS
# ---------------------------------------------------------------
for w in [7, 28]:
    df[f'wind_u_mean_w{w}'] = df['wind_u'].rolling(w, 1).mean()
    df[f'wind_v_mean_w{w}'] = df['wind_v'].rolling(w, 1).mean()
    df[f'wind_u_var_w{w}']  = df['wind_u'].rolling(w, 1).var()
    df[f'wind_v_var_w{w}']  = df['wind_v'].rolling(w, 1).var()

# ---------------------------------------------------------------
# 2️⃣  PRESSURE & HUMIDITY GRADIENTS
# ---------------------------------------------------------------
df['pressure_change_1d'] = df['sealevelpressure'].diff(1)
df['humidity_change_1d'] = df['humidity'].diff(1)
df['dew_change_1d']      = df['dew'].diff(1)

for w in [7, 28]:
    df[f'pressure_mean_w{w}'] = df['sealevelpressure'].rolling(w,1).mean()
    df[f'pressure_var_w{w}']  = df['sealevelpressure'].rolling(w,1).var()
    df[f'humidity_mean_w{w}'] = df['humidity'].rolling(w,1).mean()
    df[f'humidity_var_w{w}']  = df['humidity'].rolling(w,1).var()

# ---------------------------------------------------------------
# 3️⃣  CLIMATE & REGIME INDICATORS
# ---------------------------------------------------------------
m = df['datetime'].dt.month
df['is_wet_season']   = m.isin([5,6,7,8,9,10]).astype(int)
df['season_progress'] = np.where(m < 5, 0, np.where(m > 10, 1, (m-5)/5.0))

wet14 = df['precip'].rolling(14,1).sum()
df['soil_wetness_index'] = 1 - np.exp(-0.05 * wet14)

df['sw_monsoon_flag'] = (df['wind_v'] > 0).astype(int)
df['ne_monsoon_flag'] = (df['wind_v'] < 0).astype(int)
df['wind_monsoon_index']     = 0.7*df['wind_u'] + 0.7*df['wind_v']
df['wind_monsoon_weighted']  = df['wind_monsoon_index'] * df.get('doy_sin',1.0)

# ---------------------------------------------------------------
# 4️⃣  LOW-NOISE CATEGORICAL PATTERNS
# ---------------------------------------------------------------
sea_sector = (df['winddir'] >= 90) & (df['winddir'] <= 210)
df['is_onshore_flow']  = sea_sector.astype(int)
df['is_offshore_flow'] = (~sea_sector).astype(int)

cloud7 = df['cloudcover'].rolling(7,1).mean()
df['cloudy_spell_flag'] = (cloud7 > 0.7).astype(int)
df['clear_spell_flag']  = (cloud7 < 0.3).astype(int)

df['precip_intensity'] = df['precip'] / (df['precipcover'] + 1e-3)
df.loc[df['precip']==0, 'precip_intensity'] = 0
med_intensity = df['precip_intensity'].median()
med_gust      = df['windgust'].median()
df['is_convective_rain'] = ((df['precip_intensity'] > med_intensity) &
                            (df['windgust'] > med_gust)).astype(int)
df['is_stratiform_rain'] = ((df['precip'] > 0) &
                            (df['is_convective_rain']==0)).astype(int)
df['convective_yesterday'] = df['is_convective_rain'].shift(1).fillna(0)

hum_thr = df['humidity'].median()
rad_thr = df['solarradiation'].median()
df['regime_hot_humid']    = ((df['humidity'] > hum_thr) & (df['solarradiation'] > rad_thr)).astype(int)
df['regime_hot_drier']    = ((df['humidity'] < hum_thr) & (df['solarradiation'] > rad_thr)).astype(int)
df['regime_cloudy_humid'] = ((df['humidity'] > hum_thr) & (df['solarradiation'] < rad_thr)).astype(int)

# ---------------------------------------------------------------
# 5️⃣  ADVANCED PHYSICAL-COHERENCE FEATURES (A–I)
# ---------------------------------------------------------------
u, v = df['wind_u'], df['wind_v']
df['wind_dir_consistency_w7']  = np.sqrt(u.rolling(7,1).mean()**2 + v.rolling(7,1).mean()**2) / (
                                  df['windspeed'].rolling(7,1).mean() + 1e-3)
df['wind_dir_consistency_w28'] = np.sqrt(u.rolling(28,1).mean()**2 + v.rolling(28,1).mean()**2) / (
                                  df['windspeed'].rolling(28,1).mean() + 1e-3)

df['wind_pressure_coupling'] = df['windspeed'] * (
    df['sealevelpressure'].rolling(3,1).mean() - df['sealevelpressure'].rolling(7,1).mean()
)

df['humid_radiation_balance'] = df['humidity'].rolling(3,1).mean() * df['solarradiation'].rolling(3,1).mean()
df['humid_radiation_ratio']   = df['humidity'] / (df['solarradiation'] + 1e-3)

if 'rain_risk_combo_w7' in df.columns:
    df['rain_risk_trend_3d'] = df['rain_risk_combo_w7'] - df['rain_risk_combo_w7'].shift(3)
    df['rain_risk_yesterday'] = df['rain_risk_combo_w7'].shift(1)
else:
    df['rain_risk_trend_3d'] = 0
    df['rain_risk_yesterday'] = 0

pairs = [('humidity','solarradiation'),
         ('wind_u','wind_v'),
         ('humidity','precip'),
         ('sealevelpressure','windspeed')]
for a,b in pairs:
    df[f'{a}_{b}_cov7'] = df[a].rolling(7,1).cov(df[b])

df['mean_wind_dir_w7']  = np.rad2deg(np.arctan2(df['wind_v_mean_w7'],  df['wind_u_mean_w7']))  % 360
df['mean_wind_dir_w28'] = np.rad2deg(np.arctan2(df['wind_v_mean_w28'], df['wind_u_mean_w28'])) % 360
df['mean_wind_dir_w7_rad']  = np.deg2rad(df['mean_wind_dir_w7'])
df['mean_wind_dir_w28_rad'] = np.deg2rad(df['mean_wind_dir_w28'])
df['mean_wind_dir_w7_sin']  = np.sin(df['mean_wind_dir_w7_rad'])
df['mean_wind_dir_w7_cos']  = np.cos(df['mean_wind_dir_w7_rad'])
df['mean_wind_dir_w28_sin'] = np.sin(df['mean_wind_dir_w28_rad'])
df['mean_wind_dir_w28_cos'] = np.cos(df['mean_wind_dir_w28_rad'])

bins = np.arange(-22.5, 382.5, 45)
labels = ['N','NE','E','SE','S','SW','W','NW']
df['wind_sector_w7_cat']  = pd.cut(df['mean_wind_dir_w7'],  bins=bins, labels=labels, ordered=False)
df['wind_sector_w28_cat'] = pd.cut(df['mean_wind_dir_w28'], bins=bins, labels=labels, ordered=False)

df['precip_efficiency'] = df['precip'].rolling(3,1).sum() / (df['cloudcover'].rolling(3,1).mean() + 1e-3)
df['wind_energy']      = 0.5 * (df['windspeed'] ** 2)
df['wind_energy_anom'] = df['wind_energy'] - df['wind_energy'].rolling(56,1).mean()
df['convective_potential_index'] = df['humidity'].rolling(7,1).mean() * df['solarradiation'].rolling(7,1).mean()

# ---------------------------------------------------------------
# 6️⃣  CLEANUP & ALIGNMENT
# ---------------------------------------------------------------
# Drop optional categorical sectors before training
cat_cols = [c for c in df.columns if c.endswith('_cat')]
df = df.drop(columns=cat_cols, errors='ignore')

# Fill NaN numerics
df = df.fillna(0).iloc[max(windows):].reset_index(drop=True)

print("Feature engineering complete.")
print("Total feature count:", len(df.columns))

Feature engineering complete.
Total feature count: 94


In [10]:
# === EWMA enhancement on df (not on train/val/test yet) ===
ewma_features = ['humidity', 'dew', 'solarradiation', 'sealevelpressure',
                 'windspeed', 'precip']

ewma_configs = {
    '3d': 0.5,
    '7d': 0.3,
    '14d': 0.15
}

for col in ewma_features:
    for tag, alpha in ewma_configs.items():
        df[f'{col}_ewma_{tag}'] = df[col].ewm(alpha=alpha, adjust=False).mean()

print(" EWMA features added to df")

 EWMA features added to df


## 6. Target and split

In [11]:
horizons = {"t1":1, "t2":2, "t3":3, "t4":4, "t5":5}

for name, h in horizons.items():
    df[f"target_{name}"] = df["temp"].shift(-h)

df = df.dropna(subset=[f"target_{h}" for h in horizons]).reset_index(drop=True)

print("\nMulti-horizon targets created. Shape:", df.shape)



Multi-horizon targets created. Shape: (3873, 117)


In [12]:
split_idx = int(len(df) * 0.8)
train = df.iloc[:split_idx].copy()
test  = df.iloc[split_idx:].copy()

exclude_cols = ['datetime', 'temp'] + [f"target_{h}" for h in horizons]
feature_cols = [c for c in df.columns if c not in exclude_cols]

X_train_full = train[feature_cols].copy()
X_test_full  = test[feature_cols].copy()

y_train_dict = {h: train[f"target_{h}"].copy() for h in horizons}
y_test_dict  = {h: test[f"target_{h}"].copy()  for h in horizons}

print(f"Train: {X_train_full.shape}, Test: {X_test_full.shape}")

Train: (3098, 110), Test: (775, 110)


In [13]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def climatology_pred(train_frame, test_frame, target_col):
    train_hour = pd.to_datetime(train_frame["datetime"]).dt.hour
    test_hour  = pd.to_datetime(test_frame["datetime"]).dt.hour
    
    climo_map = (
        train_frame.assign(h=train_hour)
        .groupby("h")[target_col].mean()
        .to_dict()
    )
    gmean = train_frame[target_col].mean()
    return np.array([climo_map.get(h, gmean) for h in test_hour])

baseline_metrics = {}

for h in horizons:
    y_train_h = y_train_dict[h]
    y_test_h  = y_test_dict[h]
    
    pers = test["temp"].values
    climo = climatology_pred(train.assign(target=y_train_h),
                             test.assign(target=y_test_h),
                             target_col="target")
    
    baseline_metrics[h] = {
        "RMSE_persistence": rmse(y_test_h, pers),
        "RMSE_climatology": rmse(y_test_h, climo)
    }

print("\nBaseline computation complete.")



Baseline computation complete.


In [14]:
drop_cols = ['feelslike','feelslikemax','feelslikemin','tempmax','tempmin']

X_train_base = X_train_full.drop(columns=[c for c in drop_cols if c in X_train_full.columns], errors='ignore')
X_test_base  = X_test_full.drop(columns=[c for c in drop_cols if c in X_test_full.columns],  errors='ignore')

print("\nBase features after removing leakage columns:", X_train_base.shape)


# =====================================================================
# Function: Feature Selection for ONE Horizon
#     (Same logic as your reference block — MI → XGB → family ranking)
# =====================================================================
def select_features_like_reference(X_tr, y_tr, X_te,
                                   mi_pct=25,    # top 25% MI
                                   top_families=15,
                                   max_features=95):

    # ---------------------------------------------------------------
    # 1️⃣ Mutual Information (top 25%)
    # ---------------------------------------------------------------
    mi = mutual_info_regression(X_tr, y_tr, random_state=42)
    mi_s = pd.Series(mi, index=X_tr.columns).sort_values(ascending=False)

    cut = np.percentile(mi_s, 100 - mi_pct)
    top_feats = mi_s[mi_s > cut].index

    X_tr_fs = X_tr[top_feats]
    X_te_fs = X_te[top_feats]

    print(f"  MI → kept {len(top_feats)} features")

    # ---------------------------------------------------------------
    # 2️⃣ Lightweight XGB Importance
    # ---------------------------------------------------------------
    xgb_temp = XGBRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=None,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=3,
        reg_alpha=1,
        min_child_weight=5,
        random_state=42,
        n_jobs=-1,
        eval_metric='rmse'
    )
    xgb_temp.fit(X_tr_fs, y_tr)

    imp = pd.Series(
        xgb_temp.feature_importances_,
        index=X_tr_fs.columns
    ).sort_values(ascending=False)

    # ---------------------------------------------------------------
    # 3️⃣ Group importance by feature "family" prefix
    # ---------------------------------------------------------------
    def base(c):
        m = re.match(r'([A-Za-z_]+)_', c)
        return m.group(1) if m else c

    grp = imp.groupby(imp.index.map(base)).mean().sort_values(ascending=False)

    # ---------------------------------------------------------------
    # 4️⃣ Select top N families and up to max_features
    # ---------------------------------------------------------------
    families = grp.head(top_families).index

    sel = [f for f in imp.index if base(f) in families][:max_features]

    X_tr_sel = X_tr_fs[sel]
    X_te_sel = X_te_fs[sel]

    print(f"  Final selected features: {len(sel)}")

    # ---------------------------------------------------------------
    # 5️⃣ Retrain lightweight XGB on final subset (as in your reference)
    # ---------------------------------------------------------------
    xgb_temp.fit(X_tr_sel, y_tr)

    return X_tr_sel, X_te_sel, sel


X_train_sel_dict = {}
X_test_sel_dict  = {}
selected_features_dict = {}

for h in horizons:
    print(f"\n================ FEATURE SELECTION FOR {h} ================")

    Xtr = X_train_base.copy()
    Xte = X_test_base.copy()
    ytr = y_train_dict[h]

    X_tr_sel, X_te_sel, sel = select_features_like_reference(Xtr, ytr, Xte)

    X_train_sel_dict[h] = X_tr_sel
    X_test_sel_dict[h]  = X_te_sel
    selected_features_dict[h] = sel

    print(f"✔ Horizon {h}: {len(sel)} features")



Base features after removing leakage columns: (3098, 105)

================ FEATURE SELECTION FOR t1 ================
  MI → kept 26 features
  Final selected features: 25
✔ Horizon t1: 25 features

================ FEATURE SELECTION FOR t2 ================
  MI → kept 26 features
  Final selected features: 26
✔ Horizon t2: 26 features

================ FEATURE SELECTION FOR t3 ================
  MI → kept 26 features
  Final selected features: 26
✔ Horizon t3: 26 features

================ FEATURE SELECTION FOR t4 ================
  MI → kept 26 features
  Final selected features: 24
✔ Horizon t4: 24 features

================ FEATURE SELECTION FOR t5 ================
  MI → kept 26 features
  Final selected features: 26
✔ Horizon t5: 26 features


In [15]:
tscv = TimeSeriesSplit(n_splits=5)

def rmse_metric(y_true, y_pred):
    return rmse(y_true, y_pred)

scorer = make_scorer(lambda yt, yp: -rmse_metric(yt, yp), greater_is_better=True)

In [16]:
rf_best_params   = {}
xgb_best_params  = {}
lgbm_best_params = {}

for h in horizons:
    y_train_h = y_train_dict[h]
    X_h = X_train_sel_dict[h]

    print(f"\n🔍 OPTIMIZING HORIZON {h}")

    # RF
    def objective_rf(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 400, step=50),
            "max_depth": trial.suggest_int("max_depth", 4, 8),
            "min_samples_split": trial.suggest_int("min_samples_split", 5, 20),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 3, 10),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", 0.3, 0.5]),
            "random_state": 42,
            "n_jobs": -1
        }
        model = RandomForestRegressor(**params)
        scores = cross_val_score(model, X_h, y_train_h, cv=tscv, scoring=scorer, n_jobs=-1)
        return np.mean(scores)

    study_rf = optuna.create_study(direction="maximize", sampler=TPESampler(seed=42))
    study_rf.optimize(objective_rf, n_trials=30)
    rf_best_params[h] = study_rf.best_params

    # XGB
    def objective_xgb(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 200, 600, step=100),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 8),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 7),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 10.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 2.0),
            "random_state": 42,
            "n_jobs": -1,
            "objective": "reg:squarederror"
        }
        model = XGBRegressor(**params)
        scores = cross_val_score(model, X_h, y_train_h, cv=tscv, scoring=scorer, n_jobs=-1)
        return np.mean(scores)

    study_xgb = optuna.create_study(direction="maximize", sampler=TPESampler(seed=42))
    study_xgb.optimize(objective_xgb, n_trials=30)
    xgb_best_params[h] = study_xgb.best_params

    # LGBM
    def objective_lgbm(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 300, 700, step=100),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 20, 80),
            "max_depth": trial.suggest_int("max_depth", -1, 10),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 5.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 2.0),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 40),
            "random_state": 42,
            "n_jobs": -1
        }
        model = LGBMRegressor(**params)
        scores = cross_val_score(model, X_h, y_train_h, cv=tscv, scoring=scorer, n_jobs=-1)
        return np.mean(scores)

    study_lgbm = optuna.create_study(direction="maximize", sampler=TPESampler(seed=42))
    study_lgbm.optimize(objective_lgbm, n_trials=30)
    lgbm_best_params[h] = study_lgbm.best_params

print("\n Hyperparameter tuning complete.")


[I 2025-11-16 23:33:12,800] A new study created in memory with name: no-name-8a585ed1-48c6-448e-bb43-93875210333d



🔍 OPTIMIZING HORIZON t1


[I 2025-11-16 23:33:22,815] Trial 0 finished with value: -0.881829015657668 and parameters: {'n_estimators': 200, 'max_depth': 8, 'min_samples_split': 16, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 0 with value: -0.881829015657668.
[I 2025-11-16 23:33:31,596] Trial 1 finished with value: -0.8793071927515715 and parameters: {'n_estimators': 400, 'max_depth': 7, 'min_samples_split': 16, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 1 with value: -0.8793071927515715.
[I 2025-11-16 23:33:33,182] Trial 2 finished with value: -0.8785400519894029 and parameters: {'n_estimators': 150, 'max_depth': 4, 'min_samples_split': 9, 'min_samples_leaf': 7, 'max_features': 0.5}. Best is trial 2 with value: -0.8785400519894029.
[I 2025-11-16 23:33:33,918] Trial 3 finished with value: -0.9009140406162548 and parameters: {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 2 with value: -0.878540051989


🔍 OPTIMIZING HORIZON t2


[I 2025-11-16 23:36:32,122] Trial 0 finished with value: -1.005815990481158 and parameters: {'n_estimators': 200, 'max_depth': 8, 'min_samples_split': 16, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 0 with value: -1.005815990481158.
[I 2025-11-16 23:36:35,338] Trial 1 finished with value: -1.0044501498441674 and parameters: {'n_estimators': 400, 'max_depth': 7, 'min_samples_split': 16, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 1 with value: -1.0044501498441674.
[I 2025-11-16 23:36:36,782] Trial 2 finished with value: -1.0151441380635404 and parameters: {'n_estimators': 150, 'max_depth': 4, 'min_samples_split': 9, 'min_samples_leaf': 7, 'max_features': 0.5}. Best is trial 1 with value: -1.0044501498441674.
[I 2025-11-16 23:36:37,523] Trial 3 finished with value: -1.0114347641369688 and parameters: {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 1 with value: -1.004450149844

...


🔍 OPTIMIZING HORIZON t4


[I 2025-11-16 23:42:15,479] Trial 0 finished with value: -1.075657339901726 and parameters: {'n_estimators': 200, 'max_depth': 8, 'min_samples_split': 16, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 0 with value: -1.075657339901726.
[I 2025-11-16 23:42:18,466] Trial 1 finished with value: -1.0762871343886564 and parameters: {'n_estimators': 400, 'max_depth': 7, 'min_samples_split': 16, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 0 with value: -1.075657339901726.
[I 2025-11-16 23:42:20,343] Trial 2 finished with value: -1.087176235290479 and parameters: {'n_estimators': 150, 'max_depth': 4, 'min_samples_split': 9, 'min_samples_leaf': 7, 'max_features': 0.5}. Best is trial 0 with value: -1.075657339901726.
[I 2025-11-16 23:42:21,038] Trial 3 finished with value: -1.0834209568644688 and parameters: {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 0 with value: -1.075657339901726


🔍 OPTIMIZING HORIZON t5


[I 2025-11-16 23:45:07,667] Trial 0 finished with value: -1.0889363311098719 and parameters: {'n_estimators': 200, 'max_depth': 8, 'min_samples_split': 16, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 0 with value: -1.0889363311098719.
[I 2025-11-16 23:45:11,433] Trial 1 finished with value: -1.0826045964577913 and parameters: {'n_estimators': 400, 'max_depth': 7, 'min_samples_split': 16, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 1 with value: -1.0826045964577913.
[I 2025-11-16 23:45:13,007] Trial 2 finished with value: -1.090397443264866 and parameters: {'n_estimators': 150, 'max_depth': 4, 'min_samples_split': 9, 'min_samples_leaf': 7, 'max_features': 0.5}. Best is trial 1 with value: -1.0826045964577913.
[I 2025-11-16 23:45:13,693] Trial 3 finished with value: -1.0841938072583719 and parameters: {'n_estimators': 100, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 1 with value: -1.08260459645


 Hyperparameter tuning complete.


In [ ]:
def evaluate_metrics(y_true, y_pred, base_pers, base_climo):
    RMSE = rmse(y_true, y_pred)
    return {
        "RMSE": RMSE,
        "MAE": mean_absolute_error(y_true, y_pred),
        "R2": r2_score(y_true, y_pred),
        "Skill_vs_Persistence": 1 - RMSE / base_pers,
        "Skill_vs_Climatology": 1 - RMSE / base_climo
    }

model_choice = {
    "t1": "lgbm", "t2": "lgbm", "t3": "lgbm",
    "t4": "xgb", "t5": "rf"
}

final_models = {}
results = []

for h in horizons:
    print(f"\n🔧 FINAL TRAINING — Horizon {h}")

    Xtr_h = X_train_sel_dict[h]
    Xte_h = X_test_sel_dict[h]

    y_train_h = y_train_dict[h]
    y_test_h  = y_test_dict[h]

    choice = model_choice[h]

    if choice == "lgbm":
        model = LGBMRegressor(**lgbm_best_params[h])
    elif choice == "xgb":
        model = XGBRegressor(**xgb_best_params[h])
    else:
        model = RandomForestRegressor(**rf_best_params[h])

    model.fit(Xtr_h, y_train_h)
    y_pred_test = model.predict(Xte_h)


    final_models[h] = model

    base = baseline_metrics[h]
    met = evaluate_metrics(y_test_h, y_pred_test,
                           base["RMSE_persistence"],
                           base["RMSE_climatology"])

    results.append({"Horizon": h, "ModelType": choice.upper(), **met})

results_df = pd.DataFrame(results)
print("\n📊 Final Test Results (t1..t5):")
display(results_df.round(4))



🔧 FINAL TRAINING — Horizon t1
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000452 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6246
[LightGBM] [Info] Number of data points in the train set: 3098, number of used features: 25
[LightGBM] [Info] Start training from score 28.412072
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

,Horizon,ModelType,RMSE,MAE,R2,Skill_vs_Persistence,Skill_vs_Climatology
0,t1,LGBM,0.8162,0.6499,0.6810,0.0318,0.4517
1,t2,LGBM,0.9836,0.7956,0.5367,0.1135,0.3393
2,t3,LGBM,1.0314,0.8357,0.4904,0.1576,0.3069
3,t4,XGB,1.0492,0.8522,0.4723,0.1758,0.2946
4,t5,RF,1.0672,0.8653,0.4534,0.1577,0.2817
